**2장 – 머신러닝 프로젝트 처음부터 끝까지**

*머신러닝 부동산 회사에 오신 것을 환영합니다! 여러분이 할 작업은 캘리포니아 지역 주택의 여러 특성을 사용해 중간 가격을 예측하는 것입니다.*

*이 노트북은 2장의 모든 샘플 코드와 연습 문제 정답을 담고 있습니다.*

<table align="left">
  <td>
    <a href="https://colab.research.google.com/github/rickiepark/handson-ml3/blob/main/02_end_to_end_machine_learning_project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>
  </td>
</table>

*코랩에서 맷플롯립과 판다스 그래프에 한글을 쓰려면 다음 코드의 주석을 제거하고 실행하세요*

In [ ]:
# import sys

# # 노트북이 코랩에서 실행 중인지 체크합니다.
# if 'google.colab' in sys.modules:
#     !echo 'debconf debconf/frontend select Noninteractive' | debconf-set-selections
#     # 나눔 폰트를 설치합니다.
#     !sudo apt-get -qq -y install fonts-nanum
#     import matplotlib.font_manager as fm
#     font_files = fm.findSystemFonts(fontpaths=['/usr/share/fonts/truetype/nanum'])
#     for fpath in font_files:
#         fm.fontManager.addfont(fpath)

#     # 나눔바른고딕 폰트로 설정합니다.
#     import matplotlib.pyplot as plt
#     plt.rc('font', family='NanumBarunGothic')
#     # 마이너스 기호 표시 오류 수정
#     import matplotlib
#     matplotlib.rcParams['axes.unicode_minus'] = False

# 설정

In [ ]:
print("머신러닝 세계에 오신 것을 환영합니다!")

파이썬 3.7 또는 그 이상이 필요합니다:

In [ ]:
import sys

assert sys.version_info >= (3, 7)

Scikit-Learn ≥1.0.1이 필요합니다:

In [ ]:
from packaging import version
import sklearn

assert version.parse(sklearn.__version__) >= version.parse("1.0.1")

# 2.3 데이터 가져오기

## 2.3.5 데이터 다운로드하기

In [ ]:
# 슬라이드 20의 코드

## 2.3.6 데이터 구조 훑어 보기

In [ ]:
# head()


In [ ]:
# info()


In [ ]:
# "ocean_proximity" 열의 값 세기


In [ ]:
# describe()


다음 셀은 책에 없습니다. `images/end_to_end_project` 폴더가 없다면 이 폴더를 만들고 고해상도 이미지 저장을 위해 노트북에서 사용할 `save_fig()` 함수를 정의합니다.

In [ ]:
# 추가 코드 - 고해상도 PNG 파일로 그래프를 저장하기 위한 코드

IMAGES_PATH = Path() / "images" / "end_to_end_project"
IMAGES_PATH.mkdir(parents=True, exist_ok=True)

def save_fig(fig_id, tight_layout=True, fig_extension="png", resolution=300):
    path = IMAGES_PATH / f"{fig_id}.{fig_extension}"
    if tight_layout:
        plt.tight_layout()
    plt.savefig(path, format=fig_extension, dpi=resolution)

In [ ]:
import matplotlib.pyplot as plt

# 추가 코드 – 다음 다섯 라인은 기본 폰트 크기를 지정합니다
plt.rc('font', size=14)
plt.rc('axes', labelsize=14, titlesize=14)
plt.rc('legend', fontsize=14)
plt.rc('xtick', labelsize=10)
plt.rc('ytick', labelsize=10)

# 히스토그램 그리기


## 2.3.7 테스트 세트 만들기

In [ ]:
# shuffle_and_split_data()


In [ ]:

# len(train_set)

In [ ]:
# len(test_set)

노트북 실행 결과를 일정하게 유지하기 위해 랜덤 시드를 설정합니다:

In [ ]:
np.random.seed(42)

하지만 여러 변동 요인으로 인해 노트북이 책에 실린 것과 완전히 똑같은 출력을 만든다고 보장하지 않습니다. 가장 중요한 요소는 라이브러리가 업데이트되면서 시간이 지나면서 알고리즘이 조금씩 바뀌는 것입니다. 따라서 약간의 차이는 이해해 주세요. 아마도 대부분의 출력은 동일하거나 적어도 같은 범주에 있을 겁니다.

Note: 또 다른 무작위성 요소는 파이썬 set의 순서입니다. 파이썬 시작시에 랜덤하게 시드가 부여되는 `hash()` 함수를 기반으로 합니다(DoS 공격을 막기 위해 파이썬 3.3부터 적용되었습니다). 이런 무작위성을 제거하려면 파이썬을 시작하기 _전에_ `PYTHONHASHSEED` 환경 변수를 `"0"`으로 설정합니다. 파이썬을 시작한 후에 이를 설정하면 아무런 효과가 없습니다. 다행히 코랩에서 노트북을 실행하면 이 변수가 항상 설정됩니다.

In [ ]:
from zlib import crc32

def is_id_in_test_set(identifier, test_ratio):
    return crc32(np.int64(identifier)) < test_ratio * 2**32

def split_data_with_id_hash(data, test_ratio, id_column):
    ids = data[id_column]
    in_test_set = ids.apply(lambda id_: is_id_in_test_set(id_, test_ratio))
    return data.loc[~in_test_set], data.loc[in_test_set]

In [ ]:
housing_with_id = housing.reset_index()  # `index` 열 추가
train_set, test_set = split_data_with_id_hash(housing_with_id, 0.2, "index")

In [ ]:
housing_with_id["id"] = housing["longitude"] * 1000 + housing["latitude"]
train_set, test_set = split_data_with_id_hash(housing_with_id, 0.2, "id")

In [ ]:
# train_test_split()


In [ ]:
test_set["total_bedrooms"].isnull().sum()

인구의 여성 비율이 51.1%일 때 1,000명으로 구성된 랜덤 샘플에서 여성이 48.5% 보다 작거나 53.3% 보다 많을 확률을 계산하려면 [이항 분포](https://en.wikipedia.org/wiki/Binomial_distribution)를 사용합니다. 이항 분포의 `cdf()` 메서드는 여성의 수가 주어진 값보다 작거나 같을 확률을 반환합니다.

In [ ]:
# 추가 코드 – 나쁜 샘플을 얻을 확률 10.7%를 계산하는 방법

from scipy.stats import binom

sample_size = 1000
ratio_female = 0.511
proba_too_small = binom(sample_size, ratio_female).cdf(485 - 1)
proba_too_large = 1 - binom(sample_size, ratio_female).cdf(535)
print(proba_too_small + proba_too_large)

수학 대신 시물레이션을 해도 거의 동일한 결과를 얻을 수 있습니다:

In [ ]:
# 추가 코드 – 나쁜 샘플을 얻을 확률을 시물레이션으로 계산 하는 방법

np.random.seed(42)

samples = (np.random.rand(100_000, sample_size) < ratio_female).sum(axis=1)
((samples < 485) | (samples > 535)).mean()

In [ ]:
# pd.cut()을 사용한 소득 카테고리 생성


In [ ]:
# housing_income_cat_bar_plot 그래프를 저장합니다

save_fig("housing_income_cat_bar_plot")  # extra code
# plt.show()

In [ ]:
# StratifiedShuffleSplit



In [ ]:
# strat_train_set, strat_test_set = strat_splits[0]

하나의 계층적 분할을 얻기 위해서는 이 방법이 훨씬 간단합니다:

In [ ]:
# strat_train_set, strat_test_set = train_test_split(
#     housing, test_size=0.2, stratify=housing["income_cat"], random_state=42)

In [ ]:
# strat_test_set["income_cat"].value_counts() / len(strat_test_set)

In [ ]:
# 추가 코드 – 그림 2–10를 생성합니다.

def income_cat_proportions(data):
    return data["income_cat"].value_counts() / len(data)

train_set, test_set = train_test_split(housing, test_size=0.2, random_state=42)

compare_props = pd.DataFrame({
    "Overall %": income_cat_proportions(housing),
    "Stratified %": income_cat_proportions(strat_test_set),
    "Random %": income_cat_proportions(test_set),
}).sort_index()
compare_props.index.name = "Income Category"
compare_props["Strat. Error %"] = (compare_props["Stratified %"] /
                                   compare_props["Overall %"] - 1)
compare_props["Rand. Error %"] = (compare_props["Random %"] /
                                  compare_props["Overall %"] - 1)
(compare_props * 100).round(2)

In [ ]:
for set_ in (strat_train_set, strat_test_set):
    set_.drop("income_cat", axis=1, inplace=True)

# 2.4 데이터 이해를 위한 탐색과 시각화

In [ ]:
housing = strat_train_set.copy()

## 2.4.1 지리적 데이터 시각화

In [ ]:
# bad_visualization_plot

save_fig("bad_visualization_plot")  # extra code
# plt.show()

In [ ]:
# better_visualization_plot

save_fig("better_visualization_plot")  # extra code
# plt.show()

In [ ]:
# housing_prices_scatterplot

save_fig("housing_prices_scatterplot")  # extra code
# plt.show()

다음 셀은 이 장에 있는 첫 번째 그림을 생성합니다(이 코드는 책이 없습니다). 이 그림은 배경에 캘리포니아 이미지를 넣고, 레이블 이름을 바꾸고, 그리드를 삭제하여 이전 그림을 더 멋있게 만든 것입니다.

In [ ]:
# 추가 코드 – 이 셀은 이 장의 첫 번째 그림을 생성합니다

#  캘리포니아 이미지를 다운로드합니다
filename = "california.png"
if not (IMAGES_PATH / filename).is_file():
    homl3_root = "https://github.com/ageron/handson-ml3/raw/main/"
    url = homl3_root + "images/end_to_end_project/" + filename
    print("Downloading", filename)
    urllib.request.urlretrieve(url, IMAGES_PATH / filename)

housing_renamed = housing.rename(columns={
    "latitude": "Latitude", "longitude": "Longitude",
    "population": "Population",
    "median_house_value": "Median house value (ᴜsᴅ)"})
housing_renamed.plot(
             kind="scatter", x="Longitude", y="Latitude",
             s=housing_renamed["Population"] / 100, label="Population",
             c="Median house value (ᴜsᴅ)", cmap="jet", colorbar=True,
             legend=True, figsize=(10, 7))

california_img = plt.imread(IMAGES_PATH / filename)
axis = -124.55, -113.95, 32.45, 42.05
plt.axis(axis)
plt.imshow(california_img, extent=axis)

save_fig("california_housing_prices_plot")
plt.show()

## 2.4.2 상관관계 조사

In [ ]:
# 판다스 1.5버전부터 수치형 데이터만 포함하는지 여부를 결정하는 `numeric_only` 매개변수가 추가되었습니다.
# 이 매개변수의 기본값은 `True`입니다.
# 판다스 2.0버전에서 기본값이 `False`로 바뀌므로 명시적으로 `numeric_only=True`로 지정합니다.

# 슬라이드 37의 코드

In [ ]:
corr_matrix["median_house_value"].sort_values(ascending=False)

In [ ]:
# scatter_matrix

save_fig("scatter_matrix_plot")  # 추가 코드
# plt.show()

In [ ]:
# income_vs_house_value_scatterplot

save_fig("income_vs_house_value_scatterplot")  # 추가 코드
# plt.show()

## 2.4.3 특성 조합으로 실험

In [ ]:
# 특성 조합으로 새로운 특성 만들기 (슬라이드 44)


In [ ]:
corr_matrix = housing.corr(numeric_only=True)
corr_matrix["median_house_value"].sort_values(ascending=False)

# 2.5 머신러닝 알고리즘을 위한 데이터 준비

원본 훈련 세트로 복원하고 타깃을 분리합니다(`strat_train_set.drop()`은 지정한 열을 제외한 `strat_train_set`의 복사본을 만듭니다. `inplace=True`로 지정하지 않은 한 `strat_train_set` 자체를 수정하지 않습니다).

In [ ]:
# median_house_value를 제외한 특성 목록 (슬라이드 45)


## 2.5.1 데이터 정제

책에 소개된 세 개의 옵션은 다음과 같습니다:

```python
housing.dropna(subset=["total_bedrooms"], inplace=True)    # 옵션 1

housing.drop("total_bedrooms", axis=1)                     # 옵션 2

median = housing["total_bedrooms"].median()                # 옵션 3
housing["total_bedrooms"].fillna(median, inplace=True)
```

각 옵션에 대해 `housing`을 오염시키지 않기 위해 복사본을 만들어 사용합니다. 각 옵션의 출력을 확인하지만 NaN 값을 담고 있는 행은 필터링하겠습니다.

In [ ]:
null_rows_idx = housing.isnull().any(axis=1)
housing.loc[null_rows_idx].head()

In [ ]:
housing_option1 = housing.copy()

housing_option1.dropna(subset=["total_bedrooms"], inplace=True)  # 옵션 1

housing_option1.loc[null_rows_idx].head()

In [ ]:
housing_option2 = housing.copy()

housing_option2.drop("total_bedrooms", axis=1, inplace=True)  # 옵션 2

housing_option2.loc[null_rows_idx].head()

In [ ]:
housing_option3 = housing.copy()

median = housing["total_bedrooms"].median()
housing_option3["total_bedrooms"].fillna(median, inplace=True)  # 옵션 3

housing_option3.loc[null_rows_idx].head()

In [ ]:
# SimpleImputer


`"median"` 전략을 사용하기 위해 수치 특성을 분리합니다(`ocean_proximity` 같은 텍스트 특성에서는 계산할 수 없습니다):

In [ ]:
# housing_num


In [ ]:
# imputer.fit(housing_num)

In [ ]:
# imputer.statistics_

각 특성의 중간 값을 수동으로 계산한 것과 같은지 확인해 보세요:

In [ ]:
# housing_num.median().values

훈련 세트를 변환합니다:

In [ ]:
X = imputer.transform(housing_num)

In [ ]:
imputer.feature_names_in_

In [ ]:
housing_tr = pd.DataFrame(X, columns=housing_num.columns,
                          index=housing_num.index)

In [ ]:
housing_tr.loc[null_rows_idx].head()

In [ ]:
imputer.strategy

In [ ]:
housing_tr = pd.DataFrame(X, columns=housing_num.columns,
                          index=housing_num.index)

In [ ]:
housing_tr.loc[null_rows_idx].head()  # not shown in the book

In [ ]:
#from sklearn import set_config
#
# set_config(transform_output="pandas")  # scikit-learn >= 1.2

이상치를 삭제합니다:

In [ ]:
from sklearn.ensemble import IsolationForest

isolation_forest = IsolationForest(random_state=42)
outlier_pred = isolation_forest.fit_predict(X)

In [ ]:
outlier_pred

이상치를 삭제하고 싶다면 다음 코드를 실행하세요:

In [ ]:
#housing = housing.iloc[outlier_pred == 1]
#housing_labels = housing_labels.iloc[outlier_pred == 1]

## 2.5.2 텍스트와 범주형 특성 다루기

이제 범주형 입력 특성인 `ocean_proximity`을 전처리합니다:

In [ ]:
housing_cat = housing[["ocean_proximity"]]
housing_cat.head(8)

In [ ]:
# OrdinalEncoder


In [ ]:
housing_cat_encoded[:8]

In [ ]:
ordinal_encoder.categories_

In [ ]:
# OneHotEncoder

In [ ]:
housing_cat_1hot

`OneHotEncoder`는 기본적으로 희소 행렬을 반환합니다. 필요하면 `toarray()` 메서드를 사용해 밀집 배열로 변환할 수 있습니다:

In [ ]:
housing_cat_1hot.toarray()

또는 `OneHotEncoder`를 만들 때 `sparse=False`로 지정할 수 있습니다:

In [ ]:
# 사이킷런 1.2버전에서 `sparse_output` 매개변수가 추가되었고 `sparse` 매개변수는 1.4버전에서 삭제됩니다.
# 이에 대한 경고를 피하려면 `sparse_output`을 사용하세요.
cat_encoder = OneHotEncoder(sparse_output=False)
housing_cat_1hot = cat_encoder.fit_transform(housing_cat)
housing_cat_1hot

In [ ]:
cat_encoder.categories_

In [ ]:
df_test = pd.DataFrame({"ocean_proximity": ["INLAND", "NEAR BAY"]})
pd.get_dummies(df_test)

In [ ]:
cat_encoder.transform(df_test)

In [ ]:
df_test_unknown = pd.DataFrame({"ocean_proximity": ["<2H OCEAN", "ISLAND"]})
pd.get_dummies(df_test_unknown)

In [ ]:
cat_encoder.handle_unknown = "ignore"
cat_encoder.transform(df_test_unknown)

In [ ]:
cat_encoder.feature_names_in_

In [ ]:
cat_encoder.get_feature_names_out()

In [ ]:
df_output = pd.DataFrame(cat_encoder.transform(df_test_unknown),
                         columns=cat_encoder.get_feature_names_out(),
                         index=df_test_unknown.index)

In [ ]:
df_output

## 2.5.3 특성 스케일링

In [ ]:
# MinMaxScalar


In [ ]:
# StandardScalar


In [ ]:
# 추가 코드 – 이 셀은 그림 2–17을 생성합니다
fig, axs = plt.subplots(1, 2, figsize=(8, 3), sharey=True)
housing["population"].hist(ax=axs[0], bins=50)
housing["population"].apply(np.log).hist(ax=axs[1], bins=50)
axs[0].set_xlabel("Population")
axs[1].set_xlabel("Log of population")
axs[0].set_ylabel("Number of districts")
save_fig("long_tail_plot")
plt.show()

각 값을 백분위수로 바꾸면 어떻게 될까요?

In [ ]:
# 추가 코드 – 균등 분포를 얻는다는 것을 보여줍니다
percentiles = [np.percentile(housing["median_income"], p)
               for p in range(1, 100)]
flattened_median_income = pd.cut(housing["median_income"],
                                 bins=[-np.inf] + percentiles + [np.inf],
                                 labels=range(1, 100 + 1))
flattened_median_income.hist(bins=50)
plt.xlabel("Median income percentile")
plt.ylabel("Number of districts")
plt.show()
# Note: 1 백분위수 아래의 소득은 1로 레이블되고 99 백분위수 이상의 값은 100으로 레이블됩니다.
# 이 때문에 아래 분포의 범위가 1에서부터 100까지입니다(0에서부터 100까지가 아닙니다).

In [ ]:
# rbf_kernel (슬라이드 68)


In [ ]:
# 추가 코드 – 이 셀은 그림 2–18을 생성 합니다

ages = np.linspace(housing["housing_median_age"].min(),
                   housing["housing_median_age"].max(),
                   500).reshape(-1, 1)
gamma1 = 0.1
gamma2 = 0.03
rbf1 = rbf_kernel(ages, [[35]], gamma=gamma1)
rbf2 = rbf_kernel(ages, [[35]], gamma=gamma2)

fig, ax1 = plt.subplots()

ax1.set_xlabel("Housing median age")
ax1.set_ylabel("Number of districts")
ax1.hist(housing["housing_median_age"], bins=50)

ax2 = ax1.twinx()  # x축을 공유 하는 쌍둥이 축을 만듭니다
color = "blue"
ax2.plot(ages, rbf1, color=color, label="gamma = 0.10")
ax2.plot(ages, rbf2, color=color, label="gamma = 0.03", linestyle="--")
ax2.tick_params(axis='y', labelcolor=color)
ax2.set_ylabel("Age similarity", color=color)

plt.legend(loc="upper left")
save_fig("age_similarity_plot")
plt.show()

In [ ]:
# LinearRegression


In [ ]:
predictions

In [ ]:
# TransformedTargetRegressor


In [ ]:
predictions

## 2.5.4 사용자 정의 변환기

간단한 변환기를 만들어 보죠:

In [ ]:
# FunctionTransformer


In [ ]:
# rbf_transformer = ...

In [ ]:
age_simil_35

In [ ]:
sf_coords = 37.7749, -122.41
sf_transformer = FunctionTransformer(rbf_kernel,
                                     kw_args=dict(Y=[sf_coords], gamma=0.1))
sf_simil = sf_transformer.transform(housing[["latitude", "longitude"]])

In [ ]:
sf_simil

In [ ]:
ratio_transformer = FunctionTransformer(lambda X: X[:, [0]] / X[:, [1]])
ratio_transformer.transform(np.array([[1., 2.], [3., 4.]]))

In [ ]:
# StandardScalarClone


In [ ]:
# ClusterSimilarity


In [ ]:
# cluster_simil = ...

In [ ]:
similarities[:3].round(2)

In [ ]:
# 추가 코드 – 이 셀은 그림 2–19를 생성합니다

housing_renamed = housing.rename(columns={
    "latitude": "Latitude", "longitude": "Longitude",
    "population": "Population",
    "median_house_value": "Median house value (ᴜsᴅ)"})
housing_renamed["Max cluster similarity"] = similarities.max(axis=1)

housing_renamed.plot(kind="scatter", x="Longitude", y="Latitude", grid=True,
                     s=housing_renamed["Population"] / 100, label="Population",
                     c="Max cluster similarity",
                     cmap="jet", colorbar=True,
                     legend=True, sharex=False, figsize=(10, 7))
plt.plot(cluster_simil.kmeans_.cluster_centers_[:, 1],
         cluster_simil.kmeans_.cluster_centers_[:, 0],
         linestyle="", color="black", marker="X", markersize=20,
         label="Cluster centers")
plt.legend(loc="upper right")
save_fig("district_cluster_plot")
plt.show()

## 2.5.5 변환 파이프라인

수치 특성을 전처리하는 파이프라인을 만들어 보겠습니다:

In [ ]:
# Pipeline


In [ ]:
# make_pipeline

In [ ]:
from sklearn import set_config

set_config(display='diagram')

num_pipeline

In [ ]:
housing_num_prepared = num_pipeline.fit_transform(housing_num)
housing_num_prepared[:2].round(2)

In [ ]:
# def monkey_patch_get_signature_names_out():
#     """Monkey patch some classes which did not handle get_feature_names_out()
#        correctly in Scikit-Learn 1.0.*."""
#     from inspect import Signature, signature, Parameter
#     import pandas as pd
#     from sklearn.impute import SimpleImputer
#     from sklearn.pipeline import make_pipeline, Pipeline
#     from sklearn.preprocessing import FunctionTransformer, StandardScaler

#     default_get_feature_names_out = StandardScaler.get_feature_names_out

#     if not hasattr(SimpleImputer, "get_feature_names_out"):
#       print("Monkey-patching SimpleImputer.get_feature_names_out()")
#       SimpleImputer.get_feature_names_out = default_get_feature_names_out

#     if not hasattr(FunctionTransformer, "get_feature_names_out"):
#         print("Monkey-patching FunctionTransformer.get_feature_names_out()")
#         orig_init = FunctionTransformer.__init__
#         orig_sig = signature(orig_init)

#         def __init__(*args, feature_names_out=None, **kwargs):
#             orig_sig.bind(*args, **kwargs)
#             orig_init(*args, **kwargs)
#             args[0].feature_names_out = feature_names_out

#         __init__.__signature__ = Signature(
#             list(signature(orig_init).parameters.values()) + [
#                 Parameter("feature_names_out", Parameter.KEYWORD_ONLY)])

#         def get_feature_names_out(self, names=None):
#             if callable(self.feature_names_out):
#                 return self.feature_names_out(self, names)
#             assert self.feature_names_out == "one-to-one"
#             return default_get_feature_names_out(self, names)

#         FunctionTransformer.__init__ = __init__
#         FunctionTransformer.get_feature_names_out = get_feature_names_out

# monkey_patch_get_signature_names_out()

In [ ]:
df_housing_num_prepared = pd.DataFrame(
    housing_num_prepared, columns=num_pipeline.get_feature_names_out(),
    index=housing_num.index)

In [ ]:
df_housing_num_prepared.head(2)  # 추가 코드

In [ ]:
num_pipeline.steps

In [ ]:
num_pipeline[1]

In [ ]:
num_pipeline[:-1]

In [ ]:
num_pipeline.named_steps["simpleimputer"]

In [ ]:
num_pipeline.set_params(simpleimputer__strategy="median")

In [ ]:
# ColumnTransformer


In [ ]:
# make_column_transformer()

In [ ]:
housing_prepared = preprocessing.fit_transform(housing)

In [ ]:
# 추가 코드 – 필요하다면 데이터프레임으로 만들 수 있습니다
housing_prepared_fr = pd.DataFrame(
    housing_prepared,
    columns=preprocessing.get_feature_names_out(),
    index=housing.index)
housing_prepared_fr.head(2)

In [ ]:
# 완변한 파이프라인 (슬라이드 83)


In [ ]:
housing_prepared = preprocessing.fit_transform(housing)
housing_prepared.shape

In [ ]:
preprocessing.get_feature_names_out()

# 2.6 모델 선택과 훈련

## 2.6.1 훈련 세트에서 훈련하고 평가하기

In [ ]:
# LinearRegression

Let's try the full preprocessing pipeline on a few training instances:

In [ ]:
housing_predictions = lin_reg.predict(housing)
housing_predictions[:5].round(-2)  # -2 = 십의 자리에서 반올림

실제 값과 비교합니다:

In [ ]:
housing_labels.iloc[:5].values

In [ ]:
# 추가 코드 – 책에서 언급한 에러 비율을 계산합니다
error_ratios = housing_predictions[:5].round(-2) / housing_labels.iloc[:5].values - 1
print(", ".join([f"{100 * ratio:.1f}%" for ratio in error_ratios]))

In [ ]:
# mean_squared_error (슬라이드 86)

In [ ]:
# DecisionTreeRegressor


In [ ]:
housing_predictions = tree_reg.predict(housing)
tree_rmse = mean_squared_error(housing_labels, housing_predictions,
                              squared=False)
tree_rmse

## 2.6.2 교차 검증을 사용한 평가

In [ ]:
# cross_val_score

In [ ]:
pd.Series(tree_rmses).describe()

In [ ]:
# 추가 코드 – 선형 모델의 점수를 계산합니다
lin_rmses = -cross_val_score(lin_reg, housing, housing_labels,
                              scoring="neg_root_mean_squared_error", cv=10)
pd.Series(lin_rmses).describe()

**경고:** 다음 셀은 실행하는데 몇 분이 걸릴 수 있습니다:

In [ ]:
# RandomForestRegressor


In [ ]:
pd.Series(forest_rmses).describe()

교차 검증으로 측정한 RMSE(검증 에러)와 훈련 세트로 측정한 RMSE(훈련 에러)를 비교해 보죠:

In [ ]:
forest_reg.fit(housing, housing_labels)
housing_predictions = forest_reg.predict(housing)
forest_rmse = mean_squared_error(housing_labels, housing_predictions,
                                 squared=False)
forest_rmse

훈련 에러가 검증 에러 보다 훨씬 낮기 때문에 모델이 훈련 세트에 과대적합되었다는 의미입니다. 가능한 또 다른 이유는 훈련 데이터와 검증 데이터의 불일치입니다. 여기서는 한 데이터 셋을 섞은 다음 두 부분으로 나누었기 때문에 이 경우에 해당하지 않습니다.

# 2.7 모델 미세 튜닝

## 2.7.1 그리드 탐색

**경고:**  다음 셀은 실행하는데 몇 분이 걸릴 수 있습니다:

In [ ]:
# GridSearchCV


`full_pipeline.get_params().keys()`를 사용해 전체 하이퍼파라미터 리스트를 얻을 수 있습니다:

In [ ]:
# 추가 코드 – get_params().keys() 출력의 일부를 보여 줍니다
print(str(full_pipeline.get_params().keys())[:1000] + "...")

최상의 하이퍼파라미터 조합은 다음과 같습니다:

In [ ]:
grid_search.best_params_

In [ ]:
grid_search.best_estimator_

그리드 탐색에서 테스트한 하이퍼파라미터 조합의 점수를 확인해 보죠:

In [ ]:
cv_res = pd.DataFrame(grid_search.cv_results_)
cv_res.sort_values(by="mean_test_score", ascending=False, inplace=True)

# 추가 코드 – 데이터프레임을 깔끔하게 출력하기 위한 코드입니다
cv_res = cv_res[["param_preprocessing__geo__n_clusters",
                 "param_random_forest__max_features", "split0_test_score",
                 "split1_test_score", "split2_test_score", "mean_test_score"]]
score_cols = ["split0", "split1", "split2", "mean_test_rmse"]
cv_res.columns = ["n_clusters", "max_features"] + score_cols
cv_res[score_cols] = -cv_res[score_cols].round().astype(np.int64)

cv_res.head()

## 2.7.2 랜덤 탐색

In [ ]:
from sklearn.experimental import enable_halving_search_cv
from sklearn.model_selection import HalvingRandomSearchCV

30개(`n_iter` × `cv`)의 랜덤한 하이퍼파라미터 조합을 시도합니다:

**경고:**  다음 셀은 실행하는데 몇 분이 걸릴 수 있습니다:

In [ ]:
# RandomizedSearchCV


In [ ]:
# 추가 코드 – 랜덤 탐색 결과를 출력합니다
cv_res = pd.DataFrame(rnd_search.cv_results_)
cv_res.sort_values(by="mean_test_score", ascending=False, inplace=True)
cv_res = cv_res[["param_preprocessing__geo__n_clusters",
                 "param_random_forest__max_features", "split0_test_score",
                 "split1_test_score", "split2_test_score", "mean_test_score"]]
cv_res.columns = ["n_clusters", "max_features"] + score_cols
cv_res[score_cols] = -cv_res[score_cols].round().astype(np.int64)
cv_res.head()

**보너스 섹션: 하이퍼파라미터를 위한 샘플링 분포 선택 방법**

* `scipy.stats.randint(a, b+1)`: a~b 사이의 _이산적인_ 값을 가진 하이퍼파라미터. 이 범위의 모든 값은 동일한 확률 가집니다.
* `scipy.stats.uniform(a, b)`: 매우 비슷하지만 _연속적인_ 파라미터에 사용합니다.
* `scipy.stats.geom(1 / scale)`: 이산적인 값의 경우 주어진 스케일 안에서 샘플링하고 싶을 때 사용합니다. 예를 들어 scale=1000인 경우 대부분의 샘플은 이 범주 안에 있지만 모든 샘플 중 10% 정도는 100보다 작고, 10% 정도는 2300보다 큽니다.
* `scipy.stats.expon(scale)`: `geom`의 연속적인 버전입니다. `scale`을 가장 많이 등장할 값으로 지정합니다.
* `scipy.stats.loguniform(a, b)`: 하이퍼파라미터 값의 스케일을 어떻게 지정할지 모를 때 사용합니다. a=0.01, b=100으로 지정하면 0.01과 0.1 사이의 샘플링과 10과 100 사이의 샘플링 비율이 동일합니다.

다음은 `randint()`, `uniform()`, `geom()`, `expon()`에 대한 확률 질량 함수(이산형 변수)와 확률 밀도 함수(연속형 변수)의 그래프입니다:

In [ ]:
# 추가 코드 – 랜덤 서치에서 사용할 수 있는 몇가지 분포에 대한 그래프

from scipy.stats import randint, uniform, geom, expon

xs1 = np.arange(0, 7 + 1)
randint_distrib = randint(0, 7 + 1).pmf(xs1)

xs2 = np.linspace(0, 7, 500)
uniform_distrib = uniform(0, 7).pdf(xs2)

xs3 = np.arange(0, 7 + 1)
geom_distrib = geom(0.5).pmf(xs3)

xs4 = np.linspace(0, 7, 500)
expon_distrib = expon(scale=1).pdf(xs4)

plt.figure(figsize=(12, 7))

plt.subplot(2, 2, 1)
plt.bar(xs1, randint_distrib, label="scipy.randint(0, 7 + 1)")
plt.ylabel("Probability")
plt.legend()
plt.axis([-1, 8, 0, 0.2])

plt.subplot(2, 2, 2)
plt.fill_between(xs2, uniform_distrib, label="scipy.uniform(0, 7)")
plt.ylabel("PDF")
plt.legend()
plt.axis([-1, 8, 0, 0.2])

plt.subplot(2, 2, 3)
plt.bar(xs3, geom_distrib, label="scipy.geom(0.5)")
plt.xlabel("Hyperparameter value")
plt.ylabel("Probability")
plt.legend()
plt.axis([0, 7, 0, 1])

plt.subplot(2, 2, 4)
plt.fill_between(xs4, expon_distrib, label="scipy.expon(scale=1)")
plt.xlabel("Hyperparameter value")
plt.ylabel("PDF")
plt.legend()
plt.axis([0, 7, 0, 1])

plt.show()

다음은 `expon()`와 `loguniform()`의 확률 밀도 함수(왼쪽 열), log(X)의 확률 밀도 함수(오른쪽 열)입니다. 오른쪽 열은 하이퍼파라미터 스케일의 분포를 보여줍니다. `expon()`는 대체적으로 원하는 스케일을 따라 하이퍼파라미터를 선택하고 작은 스케일 쪽으로는 긴 꼬리를 형성합니다. 하지만 `loguniform()`는 스케일에 영향을 받지 않고 모두 동일한 확률을 가집니다:

In [ ]:
# 추가 코드 – expon와 loguniform의 차이를 보여줍니다

from scipy.stats import loguniform

xs1 = np.linspace(0, 7, 500)
expon_distrib = expon(scale=1).pdf(xs1)

log_xs2 = np.linspace(-5, 3, 500)
log_expon_distrib = np.exp(log_xs2 - np.exp(log_xs2))

xs3 = np.linspace(0.001, 1000, 500)
loguniform_distrib = loguniform(0.001, 1000).pdf(xs3)

log_xs4 = np.linspace(np.log(0.001), np.log(1000), 500)
log_loguniform_distrib = uniform(np.log(0.001), np.log(1000)).pdf(log_xs4)

plt.figure(figsize=(12, 7))

plt.subplot(2, 2, 1)
plt.fill_between(xs1, expon_distrib,
                 label="scipy.expon(scale=1)")
plt.ylabel("PDF")
plt.legend()
plt.axis([0, 7, 0, 1])

plt.subplot(2, 2, 2)
plt.fill_between(log_xs2, log_expon_distrib,
                 label="log(X) with X ~ expon")
plt.legend()
plt.axis([-5, 3, 0, 1])

plt.subplot(2, 2, 3)
plt.fill_between(xs3, loguniform_distrib,
                 label="scipy.loguniform(0.001, 1000)")
plt.xlabel("Hyperparameter value")
plt.ylabel("PDF")
plt.legend()
plt.axis([0.001, 1000, 0, 0.005])

plt.subplot(2, 2, 4)
plt.fill_between(log_xs4, log_loguniform_distrib,
                 label="log(X) with X ~ loguniform")
plt.xlabel("Log of hyperparameter value")
plt.legend()
plt.axis([-8, 1, 0, 0.2])

plt.show()

## 2.7.4 최상의 모델과 오차 분석

In [ ]:
# final_model = ...

In [ ]:
# sorted()

## 2.7.5 테스트 세트로 시스템 평가하기

In [ ]:
# X_test = ...
# y_test = ...

테스트 RMSE에 대한 95% 신뢰 구간을 계산합니다:

In [ ]:
# from scipy import stats


다음과 같이 수동으로 계산할 수도 있습니다:

In [ ]:
# 추가 코드 – RMSE에 대한 신뢰 구간 계산 방법
m = len(squared_errors)
mean = squared_errors.mean()
tscore = stats.t.ppf((1 + confidence) / 2, df=m - 1)
tmargin = tscore * squared_errors.std(ddof=1) / np.sqrt(m)
np.sqrt(mean - tmargin), np.sqrt(mean + tmargin)

또한 t-점수 대신 z-점수를 사용할 수 있습니다. 테스트 세트가 너무 작지 않기 때문에 큰 차이가 없습니다:

In [ ]:
# 추가 코드 – z-점수를 사용해 신뢰 구간 계산하기
zscore = stats.norm.ppf((1 + confidence) / 2)
zmargin = zscore * squared_errors.std(ddof=1) / np.sqrt(m)
np.sqrt(mean - zmargin), np.sqrt(mean + zmargin)

## 2.8 `joblib`를 사용한 모델 저장

최종 모델을 저장합니다:

In [ ]:
# joblib.dump()


이제 모델을 제품 환경에 배포할 수 있습니다. 예를 들어, 제품 환경에서 실행하기 위한 스크립트는 다음과 같을 것입니다:

In [ ]:
# joblib.load()


In [ ]:
predictions

`pickle`을 사용할 수도 있지만 `joblib`이 더 효율적입니다.

# 연습문제 해답

## 1.

문제: _서포트 벡터 머신 회귀(`sklearn.svm.SVR`)를 `kernel=“linear”`(하이퍼파라미터 `C`를 바꿔가며)나 `kernel=“rbf”`(하이퍼파라미터 `C`와 `gamma`를 바꿔가며) 등의 다양한 하이퍼파라미터 설정으로 시도해보세요. 서포트 벡터 머신은 대용량 데이터셋에 적용하기가 쉽지 않습니다. 따라서 훈련 세트의 처음 5,000개 샘플만 사용해 모델을 훈련하고 3-겹 교차 검증을 사용하세요. 그렇지 않으면 몇 시간이 걸릴 것입니다. 지금은 이 하이퍼파라미터가 무엇을 의미하는지 너무 신경 쓰지 마세요(궁금하다면 5장 노트북을 참고하세요). 최상의 `SVR` 모델은 무엇인가요?_

In [ ]:
from sklearn.model_selection import GridSearchCV
from sklearn.svm import SVR

param_grid = [
        {'svr__kernel': ['linear'], 'svr__C': [10., 30., 100., 300., 1000.,
                                               3000., 10000., 30000.0]},
        {'svr__kernel': ['rbf'], 'svr__C': [1.0, 3.0, 10., 30., 100., 300.,
                                            1000.0],
         'svr__gamma': [0.01, 0.03, 0.1, 0.3, 1.0, 3.0]},
    ]

svr_pipeline = Pipeline([("preprocessing", preprocessing), ("svr", SVR())])
grid_search = GridSearchCV(svr_pipeline, param_grid, cv=3,
                           scoring='neg_root_mean_squared_error')
grid_search.fit(housing.iloc[:5000], housing_labels.iloc[:5000])

(3-겹 교차 검증을 사용해 평가한) 최상의 모델이 달성한 점수는 다음과 같습니다:

In [ ]:
svr_grid_search_rmse = -grid_search.best_score_
svr_grid_search_rmse

`RandomForestRegressor`보다 훨씬 좋지 않네요(하지만 훨씬 적은 데이터로 훈련했습니다). 최상의 하이퍼파라미터를 확인해 보죠:

In [ ]:
grid_search.best_params_

선형 커널이 RBF 커널보다 성능이 나은 것 같습니다. `C`는 테스트한 것 중에 최대값이 선택되었습니다. 따라서 (작은 값들은 지우고) 더 큰 값의 `C`로 그리드서치를 다시 실행해 보아야 합니다. 아마도 더 큰 값의 `C`에서 성능이 높아질 것입니다.

## 2.

문제: _`GridSearchCV`를 `RandomizedSearchCV`로 바꿔보세요._

**경고**: 다음 셀은 실행하는데 몇 분이 걸립니다. `RandomizedSearchCV`를 만들 때 `verbose=2`로 지정하면 훈련 상세 과정을 볼 수 있습니다.

In [ ]:
from sklearn.model_selection import RandomizedSearchCV
from scipy.stats import expon, loguniform

# `expon()`, `loguniform()`와 다른 확률 분포 함수에 대해서는
# https://docs.scipy.org/doc/scipy/reference/stats.html 참조하세요.

# 노트: 커널이 "linear"일 때 gamma는 무시됩니다
param_distribs = {
        'svr__kernel': ['linear', 'rbf'],
        'svr__C': loguniform(20, 200_000),
        'svr__gamma': expon(scale=1.0),
    }

rnd_search = RandomizedSearchCV(svr_pipeline,
                                param_distributions=param_distribs,
                                n_iter=50, cv=3,
                                scoring='neg_root_mean_squared_error',
                                random_state=42)
rnd_search.fit(housing.iloc[:5000], housing_labels.iloc[:5000])

(3-겹 교차 검증을 사용해 평가한) 최상의 모델이 달성한 점수는 다음과 같습니다:

In [ ]:
svr_rnd_search_rmse = -rnd_search.best_score_
svr_rnd_search_rmse

훨씬 좋아졌지만 아직 `RandomForestRegressor`의 성능에는 못미칩니다. 최상의 하이퍼파라미터를 확인해 보죠:

In [ ]:
rnd_search.best_params_

이번에는 RBF 커널에 대해 최적의 하이퍼파라미터 조합을 찾았습니다. 보통 랜덤 서치가 같은 시간안에 그리드 서치보다 더 좋은 하이퍼파라미터를 찾습니다.

`gamma`에 대해 스케일이 1인 `expon()`을 사용했기 때문에 `RandomSearch`가 대체적으로 이 스케일의 값에 대해 탐색합니다. 샘플의 80%는 0.1과 2.3 사이에 있습니다(약 10%는 더 작은 값이고 약 10%는 더 큰 값이 됩니다):

In [ ]:
np.random.seed(42)

s = expon(scale=1).rvs(100_000)  # 100,000개 샘플을 얻습니다
((s > 0.105) & (s < 2.29)).sum() / 100_000

`C`에 대해 `loguniform()` 분포를 사용했습니다. 이는 랜덤 서치를 수행하기 전에 `C`의 최적 값에 대한 단서가 없다는 의미입니다. 이 함수는 20에서 200까지 범위와 2,000에서 20,000까지 범위를 동일한 비율로 탐색합니다.

## 3.

질문: _가장 중요한 특성을 선택하는 `SelectFromModel` 변환기를 준비 파이프라인에 추가해보세요._

이전에 정의한 전처리 파이프라인 다음과 최종 회귀 모델 전에 `RandomForestRegressor` 기반의 `SelectFromModel` 변환기를 추가하는 파이프라인을 만들어 보죠:

In [ ]:
from sklearn.feature_selection import SelectFromModel

selector_pipeline = Pipeline([
    ('preprocessing', preprocessing),
    ('selector', SelectFromModel(RandomForestRegressor(random_state=42),
                                 threshold=0.005)),  # min feature importance
    ('svr', SVR(C=rnd_search.best_params_["svr__C"],
                gamma=rnd_search.best_params_["svr__gamma"],
                kernel=rnd_search.best_params_["svr__kernel"])),
])

In [ ]:
selector_rmses = -cross_val_score(selector_pipeline,
                                  housing.iloc[:5000],
                                  housing_labels.iloc[:5000],
                                  scoring="neg_root_mean_squared_error",
                                  cv=3)
pd.Series(selector_rmses).describe()

흠, 특성 선택이 큰 도움이 되지 않는 것 같군요. 하지만 임곗값이 최적이 아니기 때문일지도 모릅니다. 랜덤 서치나 그리드 서치로 이 값을 튜닝해 보면 어떨까요?

## 4.

문제: _`fit()` 메서드 안에서 k-최근접 이웃 회귀(`sklearn.neighbors.KNeighborsRegressor`)를 훈련하고 `transform()` 메서드에서 이 모델의 예측을 반환하는 사용자 정의 변환기를 만들어 보세요. 이 변환기의 입력으로 위도와 경도를 사용하고 예측 결과를 하나의 특성으로 전처리 파이프라인에 추가하세요. 이렇게 하면 가장 가까운 구역의 중간 주택 가격에 대한 특성이 모델에 추가됩니다._

k-최근접 이웃 회귀에 국한하지 말고 어떤 회귀 모델도 받을 수 있는 변환기를 만들어 보죠. 이를 위해 `MetaEstimatorMixin`를 상속하고 생성자에서 `estimator` 매개변수를 받습니다. `fit()` 메서드는 이 추정기를 복사해서 사용하고 `feature_names_in_`를 저장해야 합니다. `MetaEstimatorMixin`는 필수 매개변수로 `estimator`가 전달되었는지 확인하고 추정기의 하이퍼파라미터를 튜닝에 사용가능하도록 `get_params()`와 `set_params()`를 업데이트합니다. 마지막으로 `get_feature_names_out()` 메서드를 만듭니다.

In [ ]:
from sklearn.neighbors import KNeighborsRegressor
from sklearn.base import MetaEstimatorMixin, clone

class FeatureFromRegressor(MetaEstimatorMixin, BaseEstimator, TransformerMixin):
    def __init__(self, estimator):
        self.estimator = estimator

    def fit(self, X, y=None):
        estimator_ = clone(self.estimator)
        estimator_.fit(X, y)
        self.estimator_ = estimator_
        self.n_features_in_ = self.estimator_.n_features_in_
        if hasattr(self.estimator, "feature_names_in_"):
            self.feature_names_in_ = self.estimator.feature_names_in_
        return self  # 항상 self를 반환해야 합니다!

    def transform(self, X):
        check_is_fitted(self)
        predictions = self.estimator_.predict(X)
        if predictions.ndim == 1:
            predictions = predictions.reshape(-1, 1)
        return predictions

    def get_feature_names_out(self, names=None):
        check_is_fitted(self)
        n_outputs = getattr(self.estimator_, "n_outputs_", 1)
        estimator_class_name = self.estimator_.__class__.__name__
        estimator_short_name = estimator_class_name.lower().replace("_", "")
        return [f"{estimator_short_name}_prediction_{i}"
                for i in range(n_outputs)]

사이킷런 API를 준수하는지 확인해 보죠:

In [ ]:
from sklearn.utils.estimator_checks import check_estimator

check_estimator(FeatureFromRegressor(KNeighborsRegressor()))

좋네요! 이제 테스트해 보죠:

In [ ]:
knn_reg = KNeighborsRegressor(n_neighbors=3, weights="distance")
knn_transformer = FeatureFromRegressor(knn_reg)
geo_features = housing[["latitude", "longitude"]]
knn_transformer.fit_transform(geo_features, housing_labels)

출력 특성의 이름이 어떻게 나오나요?

In [ ]:
knn_transformer.get_feature_names_out()

좋습니다. 이제 이 변환기를 전처리 파이프라인에 추가해 보겠습니다:

In [ ]:
from sklearn.base import clone

transformers = [(name, clone(transformer), columns)
                for name, transformer, columns in preprocessing.transformers]
geo_index = [name for name, _, _ in transformers].index("geo")
transformers[geo_index] = ("geo", knn_transformer, ["latitude", "longitude"])

new_geo_preprocessing = ColumnTransformer(transformers)

In [ ]:
new_geo_pipeline = Pipeline([
    ('preprocessing', new_geo_preprocessing),
    ('svr', SVR(C=rnd_search.best_params_["svr__C"],
                gamma=rnd_search.best_params_["svr__gamma"],
                kernel=rnd_search.best_params_["svr__kernel"])),
])

In [ ]:
new_pipe_rmses = -cross_val_score(new_geo_pipeline,
                                  housing.iloc[:5000],
                                  housing_labels.iloc[:5000],
                                  scoring="neg_root_mean_squared_error",
                                  cv=3)
pd.Series(new_pipe_rmses).describe()

이런 아주 않좋군요! 확실히 클러스터 유사도 특성이 훨씬 좋습니다. `KNeighborsRegressor` 하이퍼파라미터를 튜닝해야 할까요? 이것이 다음 연습문제입니다.

## 5.

문제: _`GridSearchCV`를 사용해 준비 단계의 옵션을 자동으로 탐색해보세요._

In [ ]:
param_distribs = {
    "preprocessing__geo__estimator__n_neighbors": range(1, 30),
    "preprocessing__geo__estimator__weights": ["distance", "uniform"],
    "svr__C": loguniform(20, 200_000),
    "svr__gamma": expon(scale=1.0),
}

new_geo_rnd_search = RandomizedSearchCV(new_geo_pipeline,
                                        param_distributions=param_distribs,
                                        n_iter=50,
                                        cv=3,
                                        scoring='neg_root_mean_squared_error',
                                        random_state=42)
new_geo_rnd_search.fit(housing.iloc[:5000], housing_labels.iloc[:5000])

In [ ]:
new_geo_rnd_search_rmse = -new_geo_rnd_search.best_score_
new_geo_rnd_search_rmse

음.. 적어도 시도는 했네요! 클러스터 유사도 특성이 확실히 KNN 특성보다 나은 것 같습니다. 하지만 둘 다 시도해 보면 어떨까요? 전체 훈련 세트로 훈련하면 도움이 될 수도 있습니다.

## 6.

문제: _6. `StandardScalerClone` 클래스를 처음부터 다시 구현하세요. 그다음 `inverse_transform()` 메서드를 추가하세요. `scaler.
inverse_transform(scaler.fit_transform(X))`를 호출하면 `X`에 매우 가까운 배열을 반환해야 합니다. 그다음 특성 이름을 지원하는 기능을 추가하세요. 입력이 데이터프레임이면 `fit()` 메서드에서 `feature_names_in_`를 설정합니다. 이 속성은 특성 이름의 넘파이 배열입니다. 마지막으로 `get_feature_names_out()` 메서드를 구현하세요. 이 메서드는 선택적인 매개변수 `input_features=None`을 가집니다. 매개변수 값이 전달되면 길이가 `n_features_in_`과 같은지 확인하고 값이 `feature_names_in_`과 같은지 확인해야 합니다. 그다음 `input_features`를 반환해야 합니다. `input_features`가 `None`인 경우 `feature_names_in_`가 정의되어 있다면 이를 반환하고 그렇지 않으면 `n_features_in_` 길이의 `np.array(["x0", "x1", ...])` 배열을 반환합니다._

In [ ]:
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.utils.validation import check_array, check_is_fitted

class StandardScalerClone(BaseEstimator, TransformerMixin):
    def __init__(self, with_mean=True):  # *args나 **kwargs를 사용하지 않습니다!
        self.with_mean = with_mean

    def fit(self, X, y=None):  # y는 사용하지 않더라고 필수입니다
        X_orig = X
        X = check_array(X)  # X가 부동소수 배열인지 확인합니다
        self.mean_ = X.mean(axis=0)
        self.scale_ = X.std(axis=0)
        self.n_features_in_ = X.shape[1]  # 모든 추정기는 fit()에서 이를 저장해야 합니다
        if hasattr(X_orig, "columns"):
            self.feature_names_in_ = np.array(X_orig.columns, dtype=object)
        return self  # 항상 self를 반환합니다!

    def transform(self, X):
        check_is_fitted(self)  #  (_가 붙은) 학습된 속성을 확인합니다
        X = check_array(X)
        if self.n_features_in_ != X.shape[1]:
            raise ValueError("Unexpected number of features")
        if self.with_mean:
            X = X - self.mean_
        return X / self.scale_

    def inverse_transform(self, X):
        check_is_fitted(self)
        X = check_array(X)
        if self.n_features_in_ != X.shape[1]:
            raise ValueError("Unexpected number of features")
        X = X * self.scale_
        return X + self.mean_ if self.with_mean else X

    def get_feature_names_out(self, input_features=None):
        if input_features is None:
            return getattr(self, "feature_names_in_",
                           [f"x{i}" for i in range(self.n_features_in_)])
        else:
            if len(input_features) != self.n_features_in_:
                raise ValueError("Invalid number of features")
            if hasattr(self, "feature_names_in_") and not np.all(
                self.feature_names_in_ == input_features
            ):
                raise ValueError("input_features ≠ feature_names_in_")
            return input_features

사용자 정의 변환기를 테스트해 보죠:

In [ ]:
from sklearn.utils.estimator_checks import check_estimator

check_estimator(StandardScalerClone())

에러가 없습니다. 시작이 좋네요. 사이킷런 API를 준수했습니다.

이제 이 변환기가 기대한 대로 동작하는지 확인해 보겠습니다:

In [ ]:
np.random.seed(42)
X = np.random.rand(1000, 3)

scaler = StandardScalerClone()
X_scaled = scaler.fit_transform(X)

assert np.allclose(X_scaled, (X - X.mean(axis=0)) / X.std(axis=0))

`with_mean=False`로 지정하면 어떨까요?

In [ ]:
scaler = StandardScalerClone(with_mean=False)
X_scaled_uncentered = scaler.fit_transform(X)

assert np.allclose(X_scaled_uncentered, X / X.std(axis=0))

거꾸로 변환하는 것은 잘 될까요?

In [ ]:
scaler = StandardScalerClone()
X_back = scaler.inverse_transform(scaler.fit_transform(X))

assert np.allclose(X, X_back)

특성 이름 출력은 잘 되나요?

In [ ]:
assert np.all(scaler.get_feature_names_out() == ["x0", "x1", "x2"])
assert np.all(scaler.get_feature_names_out(["a", "b", "c"]) == ["a", "b", "c"])

데이터프레임으로 훈련했을때 입력과 출력이 잘 되나요?

In [ ]:
df = pd.DataFrame({"a": np.random.rand(100), "b": np.random.rand(100)})
scaler = StandardScalerClone()
X_scaled = scaler.fit_transform(df)

assert np.all(scaler.feature_names_in_ == ["a", "b"])
assert np.all(scaler.get_feature_names_out() == ["a", "b"])

모두 성공입니다! 오늘은 여기까지 입니다! 😀

축하합니다! 이제 머신러닝에 대해 꽤 많은 것을 알게 되었습니다. :)